In [17]:
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import argrelextrema

# --- CONFIGURATION ---
ASSETS = {
    'EURUSD': 'EURUSD=X', 'USDCHF': 'USDCHF=X', 'GBPUSD': 'GBPUSD=X',
    'USDCAD': 'USDCAD=X', 'BTCUSD': 'BTC-USD', 'ETHUSD': 'ETH-USD',
    'XAUUSD': 'XAUUSD=X', 'XAGUSD': 'XAGUSD=X', 'SP500m': '^GSPC', 'UK100': '^FTSE'
}

TIMEFRAME_SETTINGS = {
    '1WK': ('1wk', '10y'), 'D1': ('1d', '5y'), '4H': ('4h', '2y'),
    '1H': ('1h', '2y'), 'M30': ('30m', '60d'), 'M15': ('15m', '60d')
}

# Strategy Tuning
LOOKBACK_PERIOD = 14
BODY_MULTIPLIER = 3.5
BOX_WIDTH = 30
RSI_ORDER = 2   # Sensitivity for RSI peaks/troughs


def flatten_yf_columns(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    return df


def get_fvg_and_rsi_data(df):
    """Detects FVGs and RSI Divergences."""
    df = df.copy()
    time_col = 'Datetime' if 'Datetime' in df.columns else 'Date'

    # 1. Calculate RSI
    df['RSI'] = ta.rsi(df['Close'], length=14)

    # 2. Detect Local Extrema for Divergence
    df['Price_Max'] = df.iloc[argrelextrema(
        df['Close'].values, np.greater, order=RSI_ORDER)[0]]['Close']
    df['Price_Min'] = df.iloc[argrelextrema(
        df['Close'].values, np.less, order=RSI_ORDER)[0]]['Close']
    df['RSI_Max'] = df.iloc[argrelextrema(
        df['RSI'].values, np.greater, order=RSI_ORDER)[0]]['RSI']
    df['RSI_Min'] = df.iloc[argrelextrema(
        df['RSI'].values, np.less, order=RSI_ORDER)[0]]['RSI']

    # 3. Core FVG Logic
    fvg_list = [None] * len(df)
    for i in range(2, len(df)):
        h1, l1 = df['High'].iloc[i-2], df['Low'].iloc[i-2]
        o2, c2 = df['Open'].iloc[i-1], df['Close'].iloc[i-1]
        h3, l3 = df['High'].iloc[i], df['Low'].iloc[i]

        mid_body = abs(c2 - o2)
        avg_body = abs(df['Close'] - df['Open']
                       ).iloc[max(0, i-1-LOOKBACK_PERIOD):i-1].mean()
        min_gap = mid_body * 0.10

        fvg_type, bottom, top = None, 0, 0
        if l3 > h1 and mid_body > (avg_body * BODY_MULTIPLIER) and (l3 - h1 > min_gap):
            fvg_type, bottom, top = 'bullish', h1, l3
        elif h3 < l1 and mid_body > (avg_body * BODY_MULTIPLIER) and (l1 - h3 > min_gap):
            fvg_type, bottom, top = 'bearish', l1, h3

        if fvg_type:
            # Check for historical mitigation (Close-based)
            is_mitigated = False
            check_range = min(i + BOX_WIDTH, len(df))
            for j in range(i + 1, check_range):
                if fvg_type == 'bullish' and df['Close'].iloc[j] <= top:
                    is_mitigated = True
                    break
                elif fvg_type == 'bearish' and df['Close'].iloc[j] >= bottom:
                    is_mitigated = True
                    break

            fvg_list[i] = {
                'type': fvg_type, 'bottom': bottom, 'top': top,
                'start_idx': i-1, 'start_time': df[time_col].iloc[i-1],
                'mitigated': is_mitigated
            }

    df['FVG_DATA'] = fvg_list
    return df


def plot_confluence_strategy(pair, timeframe, candle_limit=150):
    interval, period = TIMEFRAME_SETTINGS[timeframe]
    df_raw = yf.download(ASSETS[pair], period=period,
                         interval=interval, auto_adjust=False, progress=False)
    if df_raw.empty:
        return

    df = flatten_yf_columns(df_raw).reset_index()
    time_col = 'Datetime' if 'Datetime' in df.columns else 'Date'
    df = get_fvg_and_rsi_data(df)

    # Slice for plotting
    df_plot = df.tail(candle_limit).copy()
    latest_close = df['Close'].iloc[-1]

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.05, row_heights=[0.7, 0.3])

    # Add Candlesticks
    fig.add_trace(go.Candlestick(
        x=df_plot[time_col], open=df_plot["Open"], high=df_plot["High"],
        low=df_plot["Low"], close=df_plot["Close"], name=pair
    ), row=1, col=1)

    # Process FVGs and Look for RSI Signals inside them
    active_fvgs = df[df['FVG_DATA'].notnull()]

    for _, fvg_row in active_fvgs.iterrows():
        fvg = fvg_row['FVG_DATA']
        start_idx = fvg['start_idx']
        end_idx = min(start_idx + BOX_WIDTH, len(df) - 1)

        # Determine Color
        live_hit = (fvg['type'] == 'bullish' and latest_close <= fvg['top']) or \
                   (fvg['type'] == 'bearish' and latest_close >= fvg['bottom'])

        if fvg['mitigated'] or live_hit:
            color = "rgba(180, 180, 180, 0.2)"  # Grey for mitigated
        else:
            color = "rgba(0, 255, 150, 0.3)" if fvg['type'] == 'bullish' else "rgba(255, 50, 50, 0.3)"

        # Only draw if the FVG is within our plot range
        if df[time_col].iloc[start_idx] >= df_plot[time_col].iloc[0]:
            fig.add_shape(type="rect", x0=fvg['start_time'], x1=df[time_col].iloc[end_idx],
                          y0=fvg['bottom'], y1=fvg['top'], fillcolor=color, layer="below", line=dict(width=0), row=1, col=1)

            # --- SEARCH FOR RSI DIVERGENCE INSIDE THIS FVG BOX ---
            # Search within the BOX_WIDTH range
            search_df = df.iloc[start_idx: end_idx + 1]

            for k in range(1, len(search_df)):
                idx = search_df.index[k]
                prev_idx = search_df.index[k-1]

                # Check Bullish Div (Inside Bullish FVG)
                if fvg['type'] == 'bullish' and not pd.isna(df.at[idx, 'Price_Min']):
                    # Look back for the previous low to compare
                    prev_lows = df.iloc[:idx][df.iloc[:idx]
                                              ['Price_Min'].notnull()].tail(2)
                    if len(prev_lows) >= 2:
                        curr_p, prev_p = df['Close'].iloc[idx], prev_lows['Close'].iloc[-2]
                        curr_r, prev_r = df['RSI'].iloc[idx], prev_lows['RSI'].iloc[-2]

                        if curr_p < prev_p and curr_r > prev_r:  # Bullish Div
                            # Signal must be physically inside the FVG price range
                            if df['Low'].iloc[idx] <= fvg['top']:
                                fig.add_trace(go.Scatter(x=[df[time_col].iloc[idx]], y=[df['Low'].iloc[idx] * 0.998],
                                              mode='markers', marker=dict(symbol='triangle-up', size=12, color='lime'),
                                              name='FVG+BullDiv', showlegend=False), row=1, col=1)

                # Check Bearish Div (Inside Bearish FVG)
                if fvg['type'] == 'bearish' and not pd.isna(df.at[idx, 'Price_Max']):
                    prev_highs = df.iloc[:idx][df.iloc[:idx]
                                               ['Price_Max'].notnull()].tail(2)
                    if len(prev_highs) >= 2:
                        curr_p, prev_p = df['Close'].iloc[idx], prev_highs['Close'].iloc[-2]
                        curr_r, prev_r = df['RSI'].iloc[idx], prev_highs['RSI'].iloc[-2]

                        if curr_p > prev_p and curr_r < prev_r:  # Bearish Div
                            if df['High'].iloc[idx] >= fvg['bottom']:
                                fig.add_trace(go.Scatter(x=[df[time_col].iloc[idx]], y=[df['High'].iloc[idx] * 1.002],
                                              mode='markers', marker=dict(symbol='triangle-down', size=12, color='yellow'),
                                              name='FVG+BearDiv', showlegend=False), row=1, col=1)

    # Subplot 2: RSI
    fig.add_trace(go.Scatter(x=df_plot[time_col], y=df_plot['RSI'], name='RSI', line=dict(
        color='purple')), row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

    fig.update_layout(title=f"<b>{pair} {timeframe}</b> - FVG Confluence with RSI Divergence",
                      template="plotly_dark", xaxis_rangeslider_visible=False, height=800)
    fig.show()


if __name__ == "__main__":
    # Example: BTC on 1 Hour timeframe
    plot_confluence_strategy('BTCUSD', 'M15', candle_limit=200)